In [1]:
import lib.wiki_module as wiki
import lib.database_module as db
import lib.encoding_module as enc
from sklearn.externals import joblib

In [2]:
pickled_transformer = joblib.load('data/vectorizer.pkl')
category_name = 'Sandwiches'
category_dict = wiki.query_category(category_name)

In [3]:
category_id = int(category_dict['categoryid'])
pages_dict = category_dict['pages']

In [4]:
db.create_or_update_category_in_database(category_name=category_name,
                                         category_id=category_id,
                                        location = 'local_group_solution')

Connected to server localhost.


'OK'

In [5]:
pages_to_vectorize = []
text_to_vectorize = []
for page in pages_dict:
#     _, _, text, _ = wiki.query_page(pageid=str(page['pageid']))
#    print wiki.query_page(pageid=str(page['pageid']))
    text = wiki.query_page(pageid=str(page['pageid']))['text']
    db.create_or_update_page_in_database(page_id = page['pageid'], 
                                         category_id=category_id, 
                                         page_title = page["title"].replace("'",""),
                                         page_text = text,
                                         location = 'local_group_solution')
    pages_to_vectorize.append(page['pageid'])
    text_to_vectorize.append(text)


Connected to server localhost.


AttributeError: 'module' object has no attribute 'create_or_update_page_vector'

In [ ]:
encoded_pages = enc.get_page_vector(transformer = pickled_transformer,
                    clean_pages=text_to_vectorize, 
                    page_ids=pages_to_vectorize)

In [ ]:
connection, cursor = db.connect_to_postgres('local_group_solution')
#cursor.execute('DELETE FROM page_vec;')
connection.commit()

for page_id, vector in encoded_pages.items():
    sql_page_vector = """INSERT INTO page_vec (page_id, page_vector)
           VALUES ({}, ARRAY{})
           on conflict do nothing;""".format(page_id, list(vector[0]))
    cursor.execute(sql_page_vector)
    connection.commit()
cursor.close()
connection.close()

In [ ]:
encoded_category = enc.get_cat_vector(transformer = pickled_transformer,
                                      clean_pages_in_cat=text_to_vectorize, 
                                      cat=category_id)
  

In [ ]:
connection, cursor = db.connect_to_postgres('local_group_solution')
connection.commit()

for cate_id in encoded_category.keys():
    sql_cate_vector = """INSERT INTO cate_vec (category_id, category_vec)
           VALUES ({}, ARRAY{})
           on conflict do nothing;""".format(cate_id, list(encoded_category[cate_id][0]))
    cursor.execute(sql_cate_vector)
    connection.commit()
cursor.close()
connection.close()

In [ ]:
len(text_to_vectorize)

In [ ]:
text_to_vectorize[6]

In [ ]:
%timeit pickled_transformer.transform([text_to_vectorize[7]])